## Setup & Install

In [1]:
# Install ultralytics if not already installed
import subprocess
import sys

try:
    import ultralytics
    print(f"ultralytics {ultralytics.__version__} already installed")
except ImportError:
    print("Installing ultralytics...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "ultralytics"])
    print("✅ ultralytics installed")

Installing ultralytics...
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached polars-1.36.1-py3-none-any.whl.metadata (10 kB)
  Using cached polars_runtime_32-1.36.1-cp39-abi3-macosx_11_0_arm64.whl.metadata (1.5 kB)
  Using cached idna-3.11-py3-none-any.whl.metadata (8.4 kB)
  Using cached urllib3-2.6.2-py3-none-any.whl.metadata (6.6 kB)
  Using cached certifi-2025.11.12-py3-none-any.whl.metadata (2.5 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 6.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.9/37.9 MB 6.0 MB/s  0:00:06 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 5.6 MB/s  0:00:00 eta 0:00:01
Using cached polars-1.36.1-py3-none-any.whl (802 kB)
Using cached polars_runtime_32-1.36.1-cp39-abi3-macosx_11_0_arm64.whl (39.3 MB)
Using cached requests-2.32.5-py3-none-any.whl (64 kB)
Using cached idna-3.11-py3-none-any.whl (71 kB)
Using cached urllib3-2.6.2-py3-none-any.whl (131 kB)
Using cached certifi-2025.11.

In [8]:
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from ultralytics import YOLO
import yaml

sys.path.insert(0, os.path.abspath('../'))

SEED = 21
DEVICE = 'mps' if torch.backends.mps.is_available() else 'cuda' if torch.cuda.is_available() else 'cpu'
BATCH_SIZE = 32
EPOCHS = 10
IMG_SIZE = 224
MODEL_NAME = 'yolov8l-cls'  # Using YOLOv8 classification

# Get directories
NOTEBOOK_DIR = Path(os.getcwd()) / 'notebooks'
RESULTS_DIR = NOTEBOOK_DIR / 'results' / 'yolo'
DATA_DIR = Path(os.getcwd()) / 'data'

# YOLO needs data in specific structure:
# dataset/
#   train/
#     class1/
#     class2/
#   val/
#     class1/
#     class2/

torch.manual_seed(SEED)
np.random.seed(SEED)

print(f"Device: {DEVICE}")
print(f"Model: {MODEL_NAME}")
print(f"Results directory: {RESULTS_DIR}")
print(f"Data directory: {DATA_DIR}")

Device: mps
Model: yolov8l-cls
Results directory: /Users/stahlma/Desktop/01_Studium/11_Thesis/soybean/thesis_poc/notebooks/results/yolo
Data directory: /Users/stahlma/Desktop/01_Studium/11_Thesis/soybean/thesis_poc/data


## Prepare Data Structure for YOLO

YOLO requires a specific directory structure. We'll use symlinks to avoid duplicating data.

In [5]:
import shutil
from sklearn.model_selection import train_test_split

def prepare_yolo_dataset(source_dir, output_dir, class_mapping, split_ratios=(0.7, 0.15, 0.15), seed=21):
    """
    Prepare dataset in YOLO classification format.
    
    Args:
        source_dir: Original dataset directory
        output_dir: Output directory for YOLO format
        class_mapping: Dict mapping original class names to standardized names
        split_ratios: (train, val, test) split ratios
    """
    output_dir = Path(output_dir)
    source_dir = Path(source_dir)
    
    # Create split directories
    for split in ['train', 'val', 'test']:
        for class_name in class_mapping.values():
            (output_dir / split / class_name).mkdir(parents=True, exist_ok=True)
    
    # Process each class
    for orig_class, std_class in class_mapping.items():
        class_dir = source_dir / orig_class
        if not class_dir.exists():
            print(f"⚠️ Warning: {class_dir} not found")
            continue
        
        # Get all images
        images = list(class_dir.glob('*.jpg')) + list(class_dir.glob('*.png')) + list(class_dir.glob('*.jpeg'))
        
        if len(images) == 0:
            print(f"⚠️ Warning: No images found in {class_dir}")
            continue
        
        # Split
        train_ratio, val_ratio, test_ratio = split_ratios
        train_imgs, temp_imgs = train_test_split(images, train_size=train_ratio, random_state=seed)
        val_imgs, test_imgs = train_test_split(temp_imgs, train_size=val_ratio/(val_ratio+test_ratio), random_state=seed)
        
        print(f"{std_class}: {len(train_imgs)} train, {len(val_imgs)} val, {len(test_imgs)} test")
        
        # Copy or symlink files
        for split, img_list in [('train', train_imgs), ('val', val_imgs), ('test', test_imgs)]:
            for img in img_list:
                dst = output_dir / split / std_class / img.name
                if not dst.exists():
                    shutil.copy2(img, dst)  # Use copy2 to preserve metadata

# Prepare ASDID dataset
print("Preparing ASDID dataset for YOLO...")
asdid_yolo_dir = DATA_DIR / 'ASDID_yolo'
asdid_class_mapping = {
    'healthy': 'Healthy',
    'soybean_rust': 'Rust',
    'frogeye': 'Frogeye'
}

if not asdid_yolo_dir.exists():
    prepare_yolo_dataset(
        source_dir=DATA_DIR / 'ASDID',
        output_dir=asdid_yolo_dir,
        class_mapping=asdid_class_mapping,
        seed=SEED
    )
    print("✅ ASDID dataset prepared")
else:
    print("✅ ASDID dataset already prepared")

# Prepare MH dataset
print("\nPreparing MH dataset for YOLO...")
mh_yolo_dir = DATA_DIR / 'MH_yolo'
mh_class_mapping = {
    'Healthy_Soyabean': 'Healthy',
    'Soyabean_Rust': 'Rust',
    'Soyabean_Frog_Leaf_Eye': 'Frogeye'
}

if not mh_yolo_dir.exists():
    prepare_yolo_dataset(
        source_dir=DATA_DIR / 'MH-SoyaHealthVision' / 'Soyabean_Leaf_Image_Dataset',
        output_dir=mh_yolo_dir,
        class_mapping=mh_class_mapping,
        seed=SEED
    )
    print("✅ MH dataset prepared")
else:
    print("✅ MH dataset already prepared")

Preparing ASDID dataset for YOLO...
Healthy: 916 train, 197 val, 197 test
Rust: 984 train, 211 val, 212 test
Frogeye: 744 train, 159 val, 160 test
✅ ASDID dataset prepared

Preparing MH dataset for YOLO...
Healthy: 142 train, 31 val, 31 test
Rust: 596 train, 128 val, 128 test
Frogeye: 118 train, 25 val, 26 test
✅ MH dataset prepared


## Load YOLOv8 Classification Model

In [6]:
# Load pretrained YOLOv8 classification model
model = YOLO(f'{MODEL_NAME}.pt')

print(f"\nModel: {MODEL_NAME}")
print(f"Task: {model.task}")
print("Model loaded successfully!")


Model: yolov8l-cls
Task: classify
Model loaded successfully!


## Train on ASDID

In [9]:
# Train the model
results = model.train(
    data=str(asdid_yolo_dir),
    epochs=EPOCHS,
    imgsz=IMG_SIZE,
    batch=BATCH_SIZE,
    device=DEVICE,
    project=str(RESULTS_DIR),
    name='yolo_asdid',
    exist_ok=True,
    seed=SEED,
    verbose=True,
    patience=10,  # Early stopping
    save=True,
    plots=True
)

print("\n✅ Training completed!")

Ultralytics 8.3.242 🚀 Python-3.12.0 torch-2.9.1 MPS (Apple M2 Pro)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/Users/stahlma/Desktop/01_Studium/11_Thesis/soybean/thesis_poc/data/ASDID_yolo, degrees=0.0, deterministic=True, device=mps, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8l-cls.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolo_asdid, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience

## Evaluate on ASDID Test Set (In-Domain)

In [10]:
# Load best model
best_model = YOLO(str(RESULTS_DIR / 'yolo_asdid' / 'weights' / 'best.pt'))

# Validate on ASDID test set
asdid_results = best_model.val(
    data=str(asdid_yolo_dir),
    split='test',
    batch=BATCH_SIZE,
    device=DEVICE,
    plots=True
)

print("\n" + "="*60)
print("ASDID Test Set (In-Domain) Results")
print("="*60)
print(f"Top-1 Accuracy: {asdid_results.top1:.4f}")
print(f"Top-5 Accuracy: {asdid_results.top5:.4f}")
print("="*60)

Ultralytics 8.3.242 🚀 Python-3.12.0 torch-2.9.1 MPS (Apple M2 Pro)
YOLOv8l-cls summary (fused): 54 layers, 36,188,419 parameters, 0 gradients, 98.7 GFLOPs
train: /Users/stahlma/Desktop/01_Studium/11_Thesis/soybean/thesis_poc/data/ASDID_yolo/train... found 2644 images in 3 classes ✅ 
val: /Users/stahlma/Desktop/01_Studium/11_Thesis/soybean/thesis_poc/data/ASDID_yolo/val... found 567 images in 3 classes ✅ 
test: /Users/stahlma/Desktop/01_Studium/11_Thesis/soybean/thesis_poc/data/ASDID_yolo/test... found 569 images in 3 classes ✅ 
test: Fast image access ✅ (ping: 0.2±0.0 ms, read: 1520.0±960.4 MB/s, size: 1854.8 KB)
test: Scanning /Users/stahlma/Desktop/01_Studium/11_Thesis/soybean/thesis_poc/data/ASDID_yolo/test... 569 images, 0 corrupt: 100% ━━━━━━━━━━━━ 569/569 4.8Kit/s 0.1s0.1s
test: New cache created: /Users/stahlma/Desktop/01_Studium/11_Thesis/soybean/thesis_poc/data/ASDID_yolo/test.cache
               classes   top1_acc   top5_acc: 100% ━━━━━━━━━━━━ 18/18 4.8s/it 1:264.0sss
      

## Evaluate on MH Test Set (Cross-Domain)

**Critical Test**: Zero-shot transfer from ASDID to MH

In [11]:
# Validate on MH test set
mh_results = best_model.val(
    data=str(mh_yolo_dir),
    split='test',
    batch=BATCH_SIZE,
    device=DEVICE,
    plots=True
)

print("\n" + "="*60)
print("MH Test Set (Cross-Domain) Results")
print("="*60)
print(f"Top-1 Accuracy: {mh_results.top1:.4f}")
print(f"Top-5 Accuracy: {mh_results.top5:.4f}")
print("="*60)

Ultralytics 8.3.242 🚀 Python-3.12.0 torch-2.9.1 MPS (Apple M2 Pro)
train: /Users/stahlma/Desktop/01_Studium/11_Thesis/soybean/thesis_poc/data/MH_yolo/train... found 856 images in 3 classes ✅ 
val: /Users/stahlma/Desktop/01_Studium/11_Thesis/soybean/thesis_poc/data/MH_yolo/val... found 184 images in 3 classes ✅ 
test: /Users/stahlma/Desktop/01_Studium/11_Thesis/soybean/thesis_poc/data/MH_yolo/test... found 185 images in 3 classes ✅ 
test: Fast image access ✅ (ping: 0.1±0.1 ms, read: 1549.9±309.9 MB/s, size: 1898.9 KB)
test: Scanning /Users/stahlma/Desktop/01_Studium/11_Thesis/soybean/thesis_poc/data/MH_yolo/test... 185 images, 0 corrupt: 100% ━━━━━━━━━━━━ 185/185 59.2it/s 3.1s0.1s
test: /Users/stahlma/Desktop/01_Studium/11_Thesis/soybean/thesis_poc/data/MH_yolo/test/Frogeye/20240920_164244.jpg: corrupt JPEG restored and saved
test: /Users/stahlma/Desktop/01_Studium/11_Thesis/soybean/thesis_poc/data/MH_yolo/test/Frogeye/20240920_164633.jpg: corrupt JPEG restored and saved
test: /Users/st

## Detailed Evaluation with Precision, Recall, F1

Now let's compute detailed metrics like we did for other models.

In [23]:
from sklearn.metrics import confusion_matrix, classification_report, precision_score, recall_score, f1_score, accuracy_score
import seaborn as sns
from tqdm.auto import tqdm

def evaluate_yolo_detailed(model, data_dir, split='test', class_names=['Healthy', 'Rust', 'Frogeye']):
    """
    Detailed evaluation with precision, recall, F1, confusion matrix
    """
    # Load model on CPU to avoid MPS issues
    model_cpu = YOLO(model.ckpt_path if hasattr(model, 'ckpt_path') else str(RESULTS_DIR / 'yolo_asdid' / 'weights' / 'best.pt'))
    
    # Get YOLO's class names (sorted alphabetically by YOLO)
    # YOLO uses alphabetical order: Frogeye(0), Healthy(1), Rust(2)
    yolo_class_names = model_cpu.names  # This is the actual mapping
    print(f"YOLO class mapping: {yolo_class_names}")
    
    # Create mapping from YOLO indices to our expected indices
    # Our expected: Healthy(0), Rust(1), Frogeye(2)
    yolo_to_our_mapping = {}
    for yolo_idx, yolo_name in yolo_class_names.items():
        our_idx = class_names.index(yolo_name)
        yolo_to_our_mapping[yolo_idx] = our_idx
    
    print(f"Index mapping (YOLO → Our): {yolo_to_our_mapping}")
    
    data_dir = Path(data_dir) / split
    all_labels = []
    all_preds = []
    
    print(f"\nEvaluating on {split} split (running on CPU to avoid MPS issues)...")
    
    for class_idx, class_name in enumerate(class_names):
        class_dir = data_dir / class_name
        if not class_dir.exists():
            print(f"⚠️ Warning: {class_dir} not found, skipping...")
            continue
            
        images = list(class_dir.glob('*.jpg')) + list(class_dir.glob('*.png')) + list(class_dir.glob('*.jpeg'))
        
        if len(images) == 0:
            print(f"⚠️ Warning: No images found in {class_dir}, skipping...")
            continue
        
        print(f"Processing {class_name}: {len(images)} images")
        for img_path in tqdm(images, desc=f"{class_name}", leave=False):
            # Predict on CPU
            results = model_cpu.predict(str(img_path), verbose=False, device='cpu')
            yolo_pred_class = int(results[0].probs.top1)
            
            # Remap YOLO's prediction to our class order
            our_pred_class = yolo_to_our_mapping[yolo_pred_class]
            
            all_labels.append(class_idx)
            all_preds.append(our_pred_class)
    
    if len(all_labels) == 0:
        print("❌ Error: No images were evaluated!")
        return None
    
    # Compute all metrics
    cm = confusion_matrix(all_labels, all_preds)
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='macro', zero_division=0)
    recall = recall_score(all_labels, all_preds, average='macro', zero_division=0)
    f1 = f1_score(all_labels, all_preds, average='macro', zero_division=0)
    
    # Per-class metrics
    per_class_report = classification_report(all_labels, all_preds, target_names=class_names, digits=4)
    
    metrics = {
        'confusion_matrix': cm,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'per_class_report': per_class_report
    }
    
    return metrics

# Evaluate ASDID
print("\n" + "="*70)
print("Evaluating ASDID Test Set (In-Domain)")
print("="*70)
asdid_metrics = evaluate_yolo_detailed(best_model, asdid_yolo_dir, 'test')

if asdid_metrics:
    print("\nASDID Test Set Metrics:")
    print(f"  Accuracy:  {asdid_metrics['accuracy']:.4f}")
    print(f"  Precision: {asdid_metrics['precision']:.4f}")
    print(f"  Recall:    {asdid_metrics['recall']:.4f}")
    print(f"  F1 Score:  {asdid_metrics['f1']:.4f}")
    print("\nPer-class metrics:")
    print(asdid_metrics['per_class_report'])
    
    # Plot confusion matrix
    plt.figure(figsize=(8, 6))
    sns.heatmap(asdid_metrics['confusion_matrix'], annot=True, fmt='d', cmap='Blues', 
                xticklabels=['Healthy', 'Rust', 'Frogeye'], 
                yticklabels=['Healthy', 'Rust', 'Frogeye'])
    plt.title(f'{MODEL_NAME} - ASDID Test (In-Domain)')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.tight_layout()
    plt.savefig(RESULTS_DIR / f'{MODEL_NAME}_asdid_cm.png', dpi=300, bbox_inches='tight')
    plt.show()

# Evaluate MH
print("\n" + "="*70)
print("Evaluating MH Test Set (Cross-Domain)")
print("="*70)
mh_metrics = evaluate_yolo_detailed(best_model, mh_yolo_dir, 'test')

if mh_metrics:
    print("\nMH Test Set Metrics:")
    print(f"  Accuracy:  {mh_metrics['accuracy']:.4f}")
    print(f"  Precision: {mh_metrics['precision']:.4f}")
    print(f"  Recall:    {mh_metrics['recall']:.4f}")
    print(f"  F1 Score:  {mh_metrics['f1']:.4f}")
    print("\nPer-class metrics:")
    print(mh_metrics['per_class_report'])
    
    # Plot confusion matrix
    plt.figure(figsize=(8, 6))
    sns.heatmap(mh_metrics['confusion_matrix'], annot=True, fmt='d', cmap='Blues',
                xticklabels=['Healthy', 'Rust', 'Frogeye'], 
                yticklabels=['Healthy', 'Rust', 'Frogeye'])
    plt.title(f'{MODEL_NAME} - MH Test (Cross-Domain)')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.tight_layout()
    plt.savefig(RESULTS_DIR / f'{MODEL_NAME}_mh_cm.png', dpi=300, bbox_inches='tight')
    plt.show()


Evaluating ASDID Test Set (In-Domain)
YOLO class mapping: {0: 'Frogeye', 1: 'Healthy', 2: 'Rust'}
Index mapping (YOLO → Our): {0: 2, 1: 0, 2: 1}

Evaluating on test split (running on CPU to avoid MPS issues)...
Processing Healthy: 197 images


Healthy:   0%|          | 0/197 [00:00<?, ?it/s]

Processing Rust: 212 images


Rust:   0%|          | 0/212 [00:00<?, ?it/s]

Processing Frogeye: 160 images


...

Rust:   0%|          | 0/128 [00:00<?, ?it/s]

Processing Frogeye: 26 images


Frogeye:   0%|          | 0/26 [00:00<?, ?it/s]


MH Test Set Metrics:
  Accuracy:  0.7189
  Precision: 0.6327
  Recall:    0.7011
  F1 Score:  0.6562

Per-class metrics:
              precision    recall  f1-score   support

     Healthy     0.7750    1.0000    0.8732        31
        Rust     0.8598    0.7188    0.7830       128
     Frogeye     0.2632    0.3846    0.3125        26

    accuracy                         0.7189       185
   macro avg     0.6327    0.7011    0.6562       185
weighted avg     0.7617    0.7189    0.7320       185



<Figure size 800x600 with 2 Axes>

## Results Summary

In [24]:
print("\n" + "="*80)
print(f"{MODEL_NAME.upper()} - Cross-Domain Performance Summary".center(80))
print("="*80)
print(f"{'Dataset':<15} {'Accuracy':>10} {'Precision':>10} {'Recall':>10} {'F1':>10}")
print("-"*80)
print(f"{'ASDID (In)':<15} {asdid_metrics['accuracy']:>10.4f} {asdid_metrics['precision']:>10.4f} "
      f"{asdid_metrics['recall']:>10.4f} {asdid_metrics['f1']:>10.4f}")
print(f"{'MH (Cross)':<15} {mh_metrics['accuracy']:>10.4f} {mh_metrics['precision']:>10.4f} "
      f"{mh_metrics['recall']:>10.4f} {mh_metrics['f1']:>10.4f}")
print("="*80)

f1_drop = asdid_metrics['f1'] - mh_metrics['f1']
print(f"\n📊 F1 Drop (ASDID→MH): {f1_drop:.4f} ({f1_drop/asdid_metrics['f1']*100:.1f}% relative)")
print(f"\n🎯 Comparison:")
print(f"   ResNet50 MH F1:   ~0.69")
print(f"   ViT-B/16 MH F1:   ~0.72")
print(f"   {MODEL_NAME} MH F1: {mh_metrics['f1']:.4f}")

if mh_metrics['f1'] > 0.72:
    print(f"\n✅ {MODEL_NAME} outperforms both baselines!")
elif mh_metrics['f1'] > 0.69:
    print(f"\n⚠️ {MODEL_NAME} better than ResNet50 but similar to ViT")
else:
    print(f"\n❌ {MODEL_NAME} underperforms compared to baselines")

print(f"\n💾 Model saved at: {RESULTS_DIR / 'yolo_asdid' / 'weights' / 'best.pt'}")


                 YOLOV8L-CLS - Cross-Domain Performance Summary                 
Dataset           Accuracy  Precision     Recall         F1
--------------------------------------------------------------------------------
ASDID (In)          0.9947     0.9939     0.9952     0.9945
MH (Cross)          0.7189     0.6327     0.7011     0.6562

📊 F1 Drop (ASDID→MH): 0.3382 (34.0% relative)

🎯 Comparison:
   ResNet50 MH F1:   ~0.69
   ViT-B/16 MH F1:   ~0.72
   yolov8l-cls MH F1: 0.6562

❌ yolov8l-cls underperforms compared to baselines

💾 Model saved at: /Users/stahlma/Desktop/01_Studium/11_Thesis/soybean/thesis_poc/notebooks/results/yolo/yolo_asdid/weights/best.pt
